In [27]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, WebSearchTool, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel, Field
import asyncio
from IPython.display import display, Markdown
from typing import Dict
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content


In [28]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a physicist student. You are doing research about quantum computing." \
"Given the search term, you search the web for the answer. Produce a concise and clear epplanation of the results." \
"Capture the main points and explain them in simple terms." \
"Add links to the sources you used." \
"Provide no more than 3 sources.";

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini"
)

In [ ]:
message = "What is the state of quantum computing in 2025?"

with trace("Web Serch Tool Agent Execution"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In [13]:
class WebSearchItem(BaseModel):
    reason: str =  Field(description="The reason for why this search was performed.")
    query: str = Field(description="The search query used.")

class WebSearchOutput(BaseModel):
    search_items: list[WebSearchItem] = Field(description="List of search items with reasons and queries.")

planner_agent = Agent(
    name="Planner Agent",
    instructions = INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type = WebSearchOutput
)

message = "What are the top trends in artificial intelligence for 2025?"

with trace("Web Search Planner Agent Execution"):
    result = await Runner.run(planner_agent, message)

print(result.final_output)

search_items=[WebSearchItem(reason='To identify forecasted advancements and innovations in AI.', query='top trends in artificial intelligence 2025'), WebSearchItem(reason='To understand the potential impact and application areas of AI in the near future.', query='future developments in artificial intelligence 2025'), WebSearchItem(reason='To gather insights on research directions and technology growth in AI.', query='AI technology trends 2025')]


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body"""

    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("")  # Change to your verified sender
    to_email = To("")  # Change to your recipient
    content = Content("text/html", html_body)
    subject = subject

    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status","success"}


202
b''
Date: Thu, 13 Nov 2025 20:28:05 GMT
Content-Length: 0
Connection: close
Server: nginx
X-Message-Id: KQQdrm-eSoW08_7bLd4zGw
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin


